In [1]:
import fasttext

kbd_model = fasttext.load_model('../data/processed/embeddings/fasttext_skipgram_kbd_300.bin')

In [2]:
import gensim.downloader as api
import numpy as np
from gensim.models import KeyedVectors
from scipy.linalg import orthogonal_procrustes

ru_model: KeyedVectors = api.load('word2vec-ruscorpora-300')  # Русские эмбеддинги
flat_keys = {key.split('_')[0]: key for key in ru_model.index_to_key}
# ru_model.get_vector(flat_keys['великан'])

In [3]:
import pandas as pd

rus_kbd_glossary_df = pd.read_csv('../data/processed/glossary/html_cleaned/Rus-Ady_UASP.csv')
kbd_rus_glossary_df = pd.read_csv('../data/processed/glossary/html_cleaned/Ady-Ady_AP.csv')

rus_kbd_glossary_df = rus_kbd_glossary_df.rename(
    columns={
        '#name': 'word',
        'Урыс-адыгэ школ псалъалъэ (Rus-Ady) (eastern)': 'description'}
)
kbd_rus_glossary_df = kbd_rus_glossary_df.rename(
    columns={
        '#name': 'word',
        'Адыгэбзэ псалъалъэ (Ady) (eastern)': 'description'}
)

In [0]:
import nltk

glossary_rus_words = set(rus_kbd_glossary_df['word'].values)
glossary_kbd_words = set(kbd_rus_glossary_df['word'].values)
glossary_kbd_words = {word.lower().replace('i', 'I') for word in glossary_kbd_words}

In [6]:
from tqdm import tqdm

kbd_kv = KeyedVectors(vector_size=300, count=len(glossary_kbd_words))
for word in tqdm(glossary_kbd_words):
    kbd_kv.add_vector(word, kbd_model.get_word_vector(word))


100%|██████████| 27268/27268 [00:00<00:00, 57579.07it/s]


# Поиск соответствий в слов в словаре

Ищем соответствия слов русского языка для слов на кабардинском языке.

In [29]:
kbd_rus_glossary_df['word'] = kbd_rus_glossary_df['word'].apply(lambda x: x.lower().replace('i', 'I'))
kbd_rus_glossary_df['description_flat'] = kbd_rus_glossary_df['description'].apply(
    lambda x: [line for line in x.split('\n') if line][0]
)
kbd_rus_glossary_df['intersection'] = kbd_rus_glossary_df['description_flat'].apply(
    lambda x: set(nltk.word_tokenize(x.lower().replace('i', 'I'))) & glossary_rus_words
)
kbd_rus_glossary_df['intersection_count'] = kbd_rus_glossary_df['intersection'].apply(len)
kbd_rus_glossary_df = kbd_rus_glossary_df[kbd_rus_glossary_df['intersection_count'] == 1]
kbd_rus_glossary_df.head(n=1000)

,word,description,intersection,description_flat,intersection_count
0,а,\nI первая буква кабардино-черкесского алфавит...,{буква},I первая буква кабардино-черкесского алфавита,1
3,абазэхэ,абазэхэр абадзехи (одна из адыгейских этническ...,{одна},абазэхэр абадзехи (одна из адыгейских этническ...,1
6,абгъуэ,"гнездо\n/ Къуалэбзухэр, джэдкъазхэр щыкIэцI, щ...",{гнездо},гнездо,1
8,абдеж,нареч. 1. там\n/ А щIыпIэм.\nАбдеж ущымыту къы...,{там},нареч. 1. там,1
12,абджыбз,"\nстеклорез (инструмент)\n/ Абдж зэрабз, зэрыз...",{инструмент},стеклорез (инструмент),1
...,...,...,...,...,...
3171,дэпсэлъэн,(допсалъэ) неперех. гл. 1. поддержать кого-л.\...,{поддержать},(допсалъэ) неперех. гл. 1. поддержать кого-л.,1
3172,дэпсэлъеин,(допсэлъей) неперех. гл. разговаривать заносчи...,{разговаривать},(допсэлъей) неперех. гл. разговаривать заносчи...,1
3178,дэпхъэнкIэн,(депхъанкIэ) перех. гл. замести что-л. внутрь ...,{замести},(депхъанкIэ) перех. гл. замести что-л. внутрь ...,1
3179,дэпхъэнкIыкIын,(депхъэнкIыкI) перех. гл. вымести что-л. откуд...,{вымести},(депхъэнкIыкI) перех. гл. вымести что-л. откуд...,1


In [31]:
# маппинг слов: английское слово -> русское слово
bilingual_dict = [
    (kbd_word, list(rus_words)[0])
    for kbd_word, rus_words in kbd_rus_glossary_df[['word', 'intersection']].values
    if list(rus_words)[0] in flat_keys
]

# Извлекаем соответствующие эмбеддинги для каждого слова из пары
X = np.array([kbd_model.get_word_vector(kbd_word) for kbd_word, _ in bilingual_dict])
Y = np.array([ru_model[flat_keys[ru_word]] for _, ru_word in bilingual_dict])

# Вычисляем ортогональное преобразование
R, _ = orthogonal_procrustes(X, Y)

# Применяем преобразование
X_aligned = np.dot(X, R)  # в X_aligned векторы будут похожи на Y
Y_aligned = np.dot(Y, R.T)  # в Y_aligned векторы будут похожи на X

# Собираем выравненные векторы

Чтобы искать соответствия слов на противоположных языках, создадим новые мапы слово -> вектор для каждого языка.

In [33]:
# ключ - слово на русском, значение - выравненный вектор на русском
rus_aligned = KeyedVectors(vector_size=300, count=len(bilingual_dict))
for word, vector in zip([ru_word for _, ru_word in bilingual_dict], Y_aligned):
    rus_aligned.add_vector(word, vector)

# ключ - слово на кабардинском, значение - выравненный вектор на русском
kbd_aligned = KeyedVectors(vector_size=300, count=len(bilingual_dict))
for word, vector in zip([kbd_word for kbd_word, _ in bilingual_dict], X_aligned):
    kbd_aligned.add_vector(word, vector)

In [52]:
from pprint import pprint

pprint(ru_model.similar_by_vector(kbd_aligned['унэ'], topn=5))
pprint(kbd_kv.similar_by_vector(rus_aligned['дом'], topn=5))

[('дом_NOUN', 0.627297580242157),
 ('шестикомнатный_ADJ', 0.5185121297836304),
 ('омеблировывать_VERB', 0.5052497386932373),
 ('восьмиквартирный_ADJ', 0.4852362871170044),
 ('квартира_NOUN', 0.4841042459011078)]
[('унэ', 0.6272976994514465),
 ('унэбжэ', 0.5661582946777344),
 ('унэишэ', 0.5644971132278442),
 ('унэн', 0.5371354222297668),
 ('унэзэн', 0.528395414352417)]


In [54]:
pprint(ru_model.similar_by_vector(kbd_aligned['куэбжэ'], topn=5))
pprint(kbd_kv.similar_by_vector(rus_aligned['ворота'], topn=5))

[('ворота_NOUN', 0.6502230167388916),
 ('калитка_NOUN', 0.6096857786178589),
 ('дверь_NOUN', 0.579963743686676),
 ('крыльцо_NOUN', 0.5683156251907349),
 ('подъезд_NOUN', 0.5473355054855347)]
[('куэбжэ', 0.6502230167388916),
 ('куэбжэIут', 0.6338037848472595),
 ('Iуэбжэ', 0.6187616586685181),
 ('бжэ', 0.6008488535881042),
 ('куэбжэдэнэ', 0.5942080616950989)]
